```
From: https://github.com/ksatola
Version: 0.1.0
```

# Model - PM2.5 - Autoregression (AR)

## Contents

- [Autoregression (AR) modelling](#base)
- Hourly forecast
    - [Load hourly data](#data_h)
    - [Modelling (train, predict/validate)](#model_h)
- Daily forecast
    - [Load daily data](#data_d)
    - [Modelling](#model_d)

In [37]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
%autoreload 2

In [39]:
import sys
sys.path.insert(0, '../src')

In [40]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
import pandas as pd 
import numpy as np

from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot

from statsmodels.graphics.tsaplots import plot_acf

from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA

import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
from model import (
    get_pm25_data_for_modelling,
    get_best_arima_params_for_time_series,
    get_df_for_lags_columns,
    split_df_for_ts_modelling_offset,
    predict_ar
)

from measure import (
    get_model_power,
    get_rmse,
    walk_forward_ts_model_validation,
    get_mean_folds_rmse_for_n_prediction_points,
    prepare_data_for_visualization
)

from plot import (
    #plot_train_test_predicted,
    plot_observed_vs_predicted,
    plot_observations_to_predictions_relationship,
    #fit_theoretical_dist_and_plot,
    plot_observed_vs_predicted_with_error
)

from stats import (
    adfuller_test
)

from logger import logger

ImportError: cannot import name 'get_mean_folds_rmse_for_n_prediction_points' from 'measure' (../src/measure/__init__.py)

---
<a id='base'></a>

## Autoregression (AR) modelling

Autoregression modeling is a modeling technique used for time series data that assumes linear continuation of the series so that previous values in the time series can be used to predict futures values. 

Autoregression technique is similar to linear regression where, you’re taking all of the previous data points to build a model to predict a future data point using a simple linear model. With the autoregression model, your’e using previous data points and using them to predict future data point(s) but with multiple lag variables.

---
<a id='data_h'></a>

## Load hourly data

In [10]:
dfh = get_pm25_data_for_modelling('ts', 'h')
dfh.head()

common.py | 42 | get_pm25_data_for_modelling | 08-Jun-20 22:00:09 | INFO: Dataframe loaded: /Users/ksatola/Documents/git/air-pollution/agh/data/dfpm25_2008-2018_hourly.hdf
common.py | 43 | get_pm25_data_for_modelling | 08-Jun-20 22:00:09 | INFO: Dataframe size: (96388, 1)


,pm25
Datetime,
2008-01-01 01:00:00,92.0
2008-01-01 02:00:00,81.0
2008-01-01 03:00:00,73.0
2008-01-01 04:00:00,60.5
2008-01-01 05:00:00,61.0


In [11]:
df = dfh.copy()

In [12]:
model_name = 'AR'

In [13]:
# Define first past/future cutoff point in time offset (1 year of data)
cut_off_offset = 365*24 # for hourly data
#cut_off_offset = 365 # for daily data

# Predict for X points
n_pred_points = 24 # for hourly data
#n_pred_points = 7 # for daily data

# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
period = 'H' # for hourly data
#period = 'D' # for daily data

### Train test split

In [14]:
# Create train / test datasets (with the offset of cut_off_offset datapoints from the end)
df_train, df_test = split_df_for_ts_modelling_offset(data=df, cut_off_offset=cut_off_offset, period=period)

common.py | 159 | split_df_for_ts_modelling_offset | 08-Jun-20 22:00:15 | INFO: Observations: 96388
common.py | 160 | split_df_for_ts_modelling_offset | 08-Jun-20 22:00:15 | INFO: Training Observations: 87627
common.py | 161 | split_df_for_ts_modelling_offset | 08-Jun-20 22:00:15 | INFO: Testing Observations: 8760
common.py | 163 | split_df_for_ts_modelling_offset | 08-Jun-20 22:00:15 | INFO: (96388, 1), (87627, 1), (8760, 1), 96387


---
<a id='model_h'></a>

## Modelling (train, predict/validate)

In statistical time series models, fitting the model means estimating its paraneters. In case of AR model, the only parameter to estimate is number of autocorrelated lags.

In [15]:
%%time
# Train the model
model = AR(df_train)
model_fitted = model.fit()

In the above, we are simply creating a testing and training dataset and then creating and fitting our AR() model. The AR() function tries to estimate the number of lags for the prediction. Once you’ve fit the model, you can look at the chosen lag and parameters of the model using some simple print statements.

In [16]:
model_fitted

In [17]:
print(f'The lag value chose is: {model_fitted.k_ar}')

The lag value chose is: 65


In [18]:
print(f'The coefficients of the model are:\n {model_fitted.params}')

The coefficients of the model are:
 const       0.699417
L1.pm25     1.217328
L2.pm25    -0.221059
L3.pm25    -0.015971
L4.pm25    -0.037128
              ...   
L61.pm25    0.005867
L62.pm25   -0.009427
L63.pm25    0.008593
L64.pm25   -0.017767
L65.pm25    0.023927
Length: 66, dtype: float64


In [24]:
%%time
# Validate result on test
# Creates 365*60*24 models for hourly data, or 365*7 models for hourly data
fold_results = walk_forward_ts_model_validation(data=df, 
                                         col_name='pm25', 
                                         model_params=model_fitted.params[:], 
                                         cut_off_offset=cut_off_offset, 
                                         n_pred_points=n_pred_points, 
                                         n_folds=-1)
print(len(fold_results))
print(fold_results[0])

8760
                     observed  predicted      error  abs_error
Datetime                                                      
2018-01-01 01:00:00  84.90085  18.256931  66.643919  66.643919
2018-01-01 02:00:00  67.44355  15.665441  51.778109  51.778109
2018-01-01 03:00:00  76.66860  15.485031  61.183569  61.183569
2018-01-01 04:00:00  64.96090  15.694880  49.266020  49.266020
2018-01-01 05:00:00  64.14875  17.793727  46.355023  46.355023
2018-01-01 06:00:00  76.06410  19.353774  56.710326  56.710326
2018-01-01 07:00:00  69.19180  20.815613  48.376187  48.376187
2018-01-01 08:00:00  48.51735  20.968488  27.548862  27.548862
2018-01-01 09:00:00  45.92715  20.423024  25.504126  25.504126
2018-01-01 10:00:00  44.19595  18.709182  25.486768  25.486768
2018-01-01 11:00:00  39.27865  17.533684  21.744966  21.744966
2018-01-01 12:00:00  32.61625  16.494254  16.121996  16.121996
2018-01-01 13:00:00  34.09440  16.915910  17.178490  17.178490
2018-01-01 14:00:00  33.51795  17.853081  15.66486

In [ ]:
8760
                     observed  predicted      error  abs_error
Datetime                                                      
2018-01-01 01:00:00  84.90085  18.256931  66.643919  66.643919
2018-01-01 02:00:00  67.44355  15.665441  51.778109  51.778109
2018-01-01 03:00:00  76.66860  15.485031  61.183569  61.183569
2018-01-01 04:00:00  64.96090  15.694880  49.266020  49.266020
2018-01-01 05:00:00  64.14875  17.793727  46.355023  46.355023
2018-01-01 06:00:00  76.06410  19.353774  56.710326  56.710326
2018-01-01 07:00:00  69.19180  20.815613  48.376187  48.376187
2018-01-01 08:00:00  48.51735  20.968488  27.548862  27.548862
2018-01-01 09:00:00  45.92715  20.423024  25.504126  25.504126
2018-01-01 10:00:00  44.19595  18.709182  25.486768  25.486768
2018-01-01 11:00:00  39.27865  17.533684  21.744966  21.744966
2018-01-01 12:00:00  32.61625  16.494254  16.121996  16.121996
2018-01-01 13:00:00  34.09440  16.915910  17.178490  17.178490
2018-01-01 14:00:00  33.51795  17.853081  15.664869  15.664869
2018-01-01 15:00:00  41.24420  19.380832  21.863368  21.863368
2018-01-01 16:00:00  49.08765  21.370328  27.717322  27.717322
2018-01-01 17:00:00  51.24645  24.365030  26.881420  26.881420
2018-01-01 18:00:00  41.64520  27.020634  14.624566  14.624566
2018-01-01 19:00:00  40.98405  29.396926  11.587124  11.587124
2018-01-01 20:00:00  45.36865  30.724681  14.643969  14.643969
2018-01-01 21:00:00  58.24830  31.317142  26.931158  26.931158
2018-01-01 22:00:00  63.21335  30.628366  32.584984  32.584984
2018-01-01 23:00:00  78.28435  29.582203  48.702147  48.702147
2018-01-02 00:00:00  91.30400  27.710736  63.593264  63.593264
CPU times: user 21min 30s, sys: 6 s, total: 21min 36s
Wall time: 21min 47s

In [25]:
%%time
# Returns a list of mean folds RMSE for n_pred_points (starting at 1 point forecast)
res = get_mean_folds_rmse_for_n_prediction_points(fold_results=fold_results, n_pred_points=n_pred_points)
res

[autoreload of measure failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../src/measure/__init__.py", line 13, in <module>
    from .validation import (
ImportError: cann

NameError: name 'get_mean_folds_rmse_for_n_prediction_points' is not defined

In [ ]:
observed, predicted, error = prepare_data_for_visualization(fold_results=fold_results, 
                                   show_n_points_of_forecast=show_n_points_of_forecast, 
                                   n_pred_points=n_pred_points)

In [ ]:
# Zooming
start_date = '2018-11'
end_date = '2018-11-30'

In [ ]:
plot_observed_vs_predicted(observed=observed[start_date:end_date],
                           predicted=predicted[start_date:end_date],
                           num_points=cut_off_offset,
                           title=model_name,
                           label_observed='PM2.5 observed',
                           label_predicted='PM2.5 predicted',
                           show_grid = True,
                           save_path='images/pm25_obs_vs_pred_365_h_ref_simple_averageXXXREMOVE.png')

In [ ]:
# Zooming
start_date = '2018-11'
end_date = '2018-11-30'

In [ ]:
plot_observed_vs_predicted_with_error(observed=observed[start_date:end_date],
                                      predicted=predicted[start_date:end_date],
                                      error=error[start_date:end_date],
                                      num_points=cut_off_offset,
                                      title=model_name,
                                      label_observed='PM2.5 observed',
                                      label_predicted='PM2.5 predicted', 
                                      label_error='Mean RMSE predition on test data',
                                      show_grid = True,
                                      save_path='images/pm25_obs_vs_pred_365_h_ref_simple_averageXXXREMOVE.png')

In [ ]:
# Zooming
start_date = '2018-11'
end_date = '2018-11-30'

In [ ]:
plot_observed_vs_predicted_with_error(observed=observed[start_date:end_date],
                                      predicted=predicted[start_date:end_date],
                                      error=error[start_date:end_date],
                                      num_points=cut_off_offset,
                                      title=model_name,
                                      label_observed='PM2.5 observed',
                                      label_predicted='PM2.5 predicted', 
                                      label_error='Mean RMSE predition on test data',
                                      show_grid = True,
                                      save_path='images/pm25_obs_vs_pred_365_h_ref_simple_averageXXXREMOVE.png')

In [ ]:
from datetime import datetime

In [ ]:
# make in sample predictions 
#predictions = model_fitted.predict(start=len(df_train), end=len(df_train) + len(df_test) - 1, dynamic=False)
predictions = model_fitted.predict(start=datetime(2017, 1, 1), end=datetime(2019, 1, 1), dynamic=False)

In [ ]:
predictions.shape

In [ ]:
plot_observed_vs_predicted(observed=df_train, 
                           predicted=predictions, 
                           num_points=cut_off_offset*2, 
                           title=model_name,
                           label_observed='PM2.5 observed', 
                           label_predicted='PM2.5 predicted', 
                           save_path='images/pm25_obs_vs_pred_365_h_ref_simple_averageXXXREMOVE.png')

In [ ]:
predictions

In [ ]:
https://machinelearningmastery.com/make-manual-predictions-arima-models-python/

In [ ]:
# Make out of sample predictions (on a test set)



In [ ]:
get_df_for_lags_columns(data=df, col_name='pm25', n_lags=10, remove_nans=True).head(10+1)

In [ ]:
model = ARIMA(df_train, order=(1,0,0))
model_fit = model.fit()
ar_coef = model_fit.arparams
predict(ar_coef, df_test)

In [ ]:
#history = [x for x in df_train]
#print(history)
predictions = []

for t in range(len(df_test)):
    model = ARIMA(df_train, order=(1,0,0))
    model_fit = model.fit()
    ar_coef = model_fit.arparams
    
    yhat = predict(ar_coef, history)
    predictions.append(yhat)
    obs = df_test[t]
    history.append(obs)
    print('>predicted=%.3f, expected=%.3f' % (yhat, obs))

rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
model.params

In [ ]:
series = df_train['pm25']
test_series = df_test['pm25']

print(series)

model_fitted = ARIMA(series, order).fit()
print(model_fitted.params)

test_model = ARIMA(test_series, order).fit()
test_model.predict()#, start=0, end=len(test_series)-1)

In [ ]:
#%%time
# Make out of sample predictions (on a test set)
#p = model_fitted.k_ar
#p = 3
order = (3, 0, 0)
train_model = ARIMA(df_train, order=order).fit()
test_model = ARIMA(df_test, order=order).fit()
#predictions = 
test_model.predict(test_model.params)

In [ ]:
df_test['pm25'].head()

In [ ]:
predictions.head()

In [ ]:
(df_test.shape, predictions.shape)

In [ ]:
plot_observed_vs_predicted(observed=df_test, 
                           predicted=predictions, 
                           num_points=cut_off_offset, 
                           title=model_name,
                           label_observed='PM2.5 observed', 
                           label_predicted='PM2.5 predicted', 
                           save_path='images/pm25_obs_vs_pred_365_h_ref_simple_averageXXXXXXDELETE.png')

---
<a id='model_h'></a>

## Modellingxxx

In statistical time series models, fitting the model means estimating its paraneters. In case of AR model, the only parameter to estimate is number of autocorrelated lags

In [ ]:
%%time
# Find best parameters (grid-search)
best_results = get_best_arima_params_for_time_series(data=df, 
                                                     seasonal=False, 
                                                     max_param_range_p=5, 
                                                     max_param_range_d=0, 
                                                     max_param_range_q=0) # AR model

In [ ]:
best_results

In [ ]:
SARIMAX(5, 0, 0) - AIC:687066.8937108214
Best model is ARIMA(5, 0, 0) with AIC of 687066.8937108214
CPU times: user 20.8 s, sys: 2.4 s, total: 23.2 s
Wall time: 8.14 s

In [ ]:
# Estimating model

# Fit an AR(5) model
model = ARMA(df, order=(5,0))
result = model.fit()  # use fit() to estimate model

# Estimated parameters
print(result.summary())

# True parameters
print(f'True parameters:\n{result.params}')

**Estimated parameters are close to true parameters. All AR coefficients are statistically significant.**

In [ ]:
df.tail(24)

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMAResults.predict.html#statsmodels.tsa.arima_model.ARMAResults.predict
# dynamic = use in-sample predictions
pred = result.predict(start='2018-12-31 01:00:00', end='2019-01-01 00:00:00', dynamic=False)
pred

In [ ]:
# Forecasting (in sample)
# Start the forecast 24 data points before the end of the point series

result.plot_predict(start='2018-12-31 01:00:00', end='2019-01-01 00:00:00', dynamic=False)
plt.show();

In [ ]:
# Forecasting (out of sample)
forecast = result.forecast(24)
plt.plot(forecast[0])
#plt.ylim(0, 40)
plt.show();

In [ ]:
observed = df.iloc[-24:, 0]
predicted = pred

plot_observed_vs_predicted(observed=observed, 
                           predicted=predicted, 
                           num_points=125,
                           title="AR Forecast",
                           label_observed='Observations', 
                           label_predicted='Predictions')

In [ ]:
plot_observations_to_predictions_relationship(observed=observed, 
                                         predicted=predicted, 
                                         title="AR(5)",
                                         label_observed='Observations',
                                         label_predicted='Predictions')

In [ ]:
rmse, r = get_model_power(df.iloc[-24:, 0], pred)
print(f'Naive forecast RMSE: {rmse:.4f}')
print(f'Naive forecast correlation coefficient of the observed-to-predicted values percent change: {r:.4f}')

---
<a id='data_d'></a>

## Load daily data

In [ ]:
data_file = data_path + 'dfpm25_2008-2018_daily.hdf'

df = pd.read_hdf(path_or_buf=data_file, key="df")
print(f'Dataframe size: {df.shape}')
df.head()

---
<a id='diag_d'></a>

## Diagnostics

In [ ]:
# Plot the time series against its lag
lag_plot(df)
plt.show();

In [ ]:
# Calculate autocorrelation coefficient
#values = pd.DataFrame(vacation.values)
df1 = pd.concat([df, df.shift(1)], axis=1)
df1.columns = ['t', 't-1']
#df1.head()
df1.corr(method='pearson')

In [ ]:
# Plot the autocorrelation plot of the dataset
plt.figure(figsize=(10, 6))
autocorrelation_plot(df)
plt.show();

In [ ]:
# Plot the AutoCorrelation Function, using candle sticks
plt.figure(figsize=(12, 6))
fig, (ax) = plt.subplots(1, 1, figsize=(20, 8))
plot_acf(df, ax=ax, lags=200)
plt.show();

---
<a id='model_d'></a>

## Modelling

In [ ]:
%%time
# Find best parameters (grid-search)
best_results = get_best_arima_params_for_time_series(data=df, 
                                                     seasonal=False, 
                                                     max_param_range_p=5, 
                                                     max_param_range_d=0, 
                                                     max_param_range_q=0) # AR model

In [ ]:
best_results

In [ ]:
# Estimating model

# Fit an AR(5) model
model = ARMA(df, order=(5,0))
result = model.fit()  # use fit() to estimate model

# Estimated parameters
print(result.summary())

# True parameters
print(f'True parameters:{result.params}')

**Estimated parameters are close to true parameters. All AR coefficients but L4 are statistically significant.**

In [ ]:
df.tail(7)

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMAResults.predict.html#statsmodels.tsa.arima_model.ARMAResults.predict
# dynamic = use in-sample predictions
pred = result.predict(start='2018-12-26', end='2019-01-01', dynamic=False)
pred

In [ ]:
# Forecasting (in-sample)
# Start the forecast 7 data points before the end of the point series

result.plot_predict(start='2018-12-26', end='2019-01-08', dynamic=False)
plt.show();

In [ ]:
# Forecasting (out of sample)
forecast = result.forecast(7)
plt.plot(forecast[0])
#plt.ylim(0, 40)
plt.show();

In [ ]:
observed = df.iloc[-7:, 0]
predicted = pred

plot_observed_vs_predicted(observed=observed, 
                           predicted=predicted, 
                           num_points=125,
                           title="AR Forecast",
                           label_observed='Observations', 
                           label_predicted='Predictions')

In [ ]:
plot_observations_to_predictions_relationship(observed=observed, 
                                         predicted=predicted, 
                                         title="AR(5)",
                                         label_observed='Observations',
                                         label_predicted='Predictions')

In [ ]:
rmse, r = get_model_power(df.iloc[-7:, 0], pred)
print(f'Naive forecast RMSE: {rmse:.4f}')
print(f'Naive forecast correlation coefficient of the observed-to-predicted values percent change: {r:.4f}')